# Classificador Valência em Tweets
Este Notebook contém os resultados de uma das etapas do Projeto Final da disciplina de "IA369-Y" na UNICAMP. São apresentados aqui os resultados da implementação de um classificador de valência (1 a 7) em tweets utilizando uma abordagem de aprendizado de máquina, especificamente, 

## Dataset 
O conjunto de dados utilizados nesta implementação são referentes aos tweets importados pela API [Tweepy](http://www.tweepy.org/) e pré-processados. O dataset é composto com as seguintes colunas:

1. id - Usuário no Twitter
2. txt - O conteúdo do tweet
3. val - Valência de 1 a 7
4. int - Intensidade de 1 a 7
5. cit - Cidade e País do usuário
6. data - Data e hora do tweet
7. dia - Dia da semana

1318 amostras x 7 colunas

Para o tratamento dos emojis, foram gerados datasets com diferentes tratamentos:

1. **output-pleasure-arousal-labeled**- Os emojis presentes nos tweets são excluídos
2. **output-pleasure-arousal-labeled-emoji** - Os emojis são mantidos, porém sem tratamento, foi observado que em uma das etapas eles são desconsiderados
3. **demojized_emojios** - Os emojis são tratados pela biblioteca [Demojize](https://github.com/nkmrtty/demojize.py/blob/master/demojize.py), ou seja, são transcritos, por exemplo *:smirking_face:* e *:yellow_heart:*

Ao final, o dataset selecionado foi o **demojized_emojios**, pois foi possível incluir os emojis na abordagem adotada pelo classificador. Entendemos que emojis, principalmente nos tweets, carregam informações valiosas sobre a valência e intensidade.

## Linguagens e Bibliotecas

1. Ambiente: [Anaconda3 4.3.1](https://repo.continuum.io/archive/index.html)
2. Linguagem de Programação: [Python 3.3](https://www.python.org/) 
3. Biblioteca de Dataframe: [Panda 0.19.2](http://pandas.pydata.org/).
4. Machine Learning: [Scikit-learn](http://scikit-learn.org/stable/index.html)
5. Plotting: [Matplotlib](https://matplotlib.org/)

## Abordagem
O dataset foi pré-processado, foram extraídas *features* de texto, separados em dados de treinamento e teste, e por fim, implementado um algoritmo de *Supporte Vector Machines* (SVM), conforme detalhes abaixo:

In [1]:
# encoding: utf-8
# encoding: iso-8859-1
# encoding: win-1252

import pandas as pd
import csv
from datetime import datetime
from dateutil import parser
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns 


## Pré-processamento
As etapas adotadas para pré-processar o dataset foram:

1. Separação de Data e Hora
2. Substituição das Cidades por Valores
3. Criação de Features [É madrugada e É final de semana]
4. Balanceamento das amostras
5. Pré-processamento dos tweets
6. Tokenizing
7. Remoção das StopWords

### Dataset com Demojize

In [44]:
### Com Demogize
datasetInFrame = pd.read_csv("demojized_emojios.csv", sep="|",quoting=csv.QUOTE_ALL)
### Colocando espaços entre os emojis
sentencesEmojis = [str(sentence).lower().replace(":"," ") for sentence in datasetInFrame["txt"]]
datasetInFrame["txt"] = sentencesEmojis
datasetInFrame.shape

(1318, 7)

### Dataset com Emoji

In [45]:
### Com emoji
#datasetInFrame = pd.read_csv("output-pleasure-arousal-labeled-emoji.csv", sep="|",quoting=csv.QUOTE_ALL)

### Dataset sem Emoji

In [46]:
### Sem emoji
#datasetInFrame = pd.read_csv("output-pleasure-arousal-labeled.csv", sep="|", encoding="ISO-8859-1",quoting=csv.QUOTE_ALL)

In [47]:
datasetInFrame.head()

,id,txt,val,int,cit,data,dia
0,line_fdl,"tô tão pensativa ultimamente, nn tô me sentind...",3,3,"Rio de Janeiro, Brasil",10/27/2017 0:17:49,6
1,line_fdl,guilherme só aparece aqui em casa p fazer barb...,5,4,"Rio de Janeiro, Brasil",10/26/2017 20:47:35,5
2,line_fdl,amo cheiro de café pela manhã.. hot_beverage ...,5,4,"Rio de Janeiro, Brasil",10/26/2017 9:45:33,5
3,line_fdl,tô meio aérea,4,2,"Rio de Janeiro, Brasil",10/26/2017 1:24:55,5
4,line_fdl,"tô mega cansada nn sei que cançaso é esse, só ...",2,4,"Rio de Janeiro, Brasil",10/26/2017 0:56:21,5


### Distribuição das Classes do Dataset
- Valência (val): 1 a 7
- Intensidade (int): 1 a 7

#### Valência

In [48]:
totalValClasses = {}

for i in range(1,8):
    totalValClasses[i] = datasetInFrame[datasetInFrame.val == i]["txt"].count()
totalValClasses

{1: 49, 2: 189, 3: 316, 4: 225, 5: 413, 6: 106, 7: 20}

In [49]:
sns.countplot(x='val', data=datasetInFrame)

#### Intensidade

In [50]:
totalIntClasses = {}

for i in range(1,8):
    totalIntClasses[i] = datasetInFrame[datasetInFrame.val == i]["txt"].count()
totalIntClasses

{1: 49, 2: 189, 3: 316, 4: 225, 5: 413, 6: 106, 7: 20}

In [51]:
sns.countplot(x='int', data=datasetInFrame)

### Separação de data e hora
Para gerar *features* posteriormente, foi adotada a estratégia separar os dados de data e hora.

In [52]:
dateTime = datasetInFrame["data"].apply(lambda  x: x.split(' '))
date = dateTime.apply(lambda x: x[0])
time = dateTime.apply(lambda x: x[1])
datasetInFrame["date"] = date
datasetInFrame["time"] = time
del datasetInFrame["data"]
datasetInFrame.head()

,id,txt,val,int,cit,dia,date,time
0,line_fdl,"tô tão pensativa ultimamente, nn tô me sentind...",3,3,"Rio de Janeiro, Brasil",6,10/27/2017,0:17:49
1,line_fdl,guilherme só aparece aqui em casa p fazer barb...,5,4,"Rio de Janeiro, Brasil",5,10/26/2017,20:47:35
2,line_fdl,amo cheiro de café pela manhã.. hot_beverage ...,5,4,"Rio de Janeiro, Brasil",5,10/26/2017,9:45:33
3,line_fdl,tô meio aérea,4,2,"Rio de Janeiro, Brasil",5,10/26/2017,1:24:55
4,line_fdl,"tô mega cansada nn sei que cançaso é esse, só ...",2,4,"Rio de Janeiro, Brasil",5,10/26/2017,0:56:21


### Substituição de Valores Para Cidade
1. Removendo o País;
2. Substituindo: 
    * Rio de Janeiro = 1 
    * São Paulo = 2

Essa estratégia foi adotada para incluir a cidade como uma *feature* no classificador. Apesar desse processamento, essa coluna não foi utilizada na implementação do modelo apresentado aqui.

In [53]:
city = datasetInFrame["cit"].apply(lambda x: x.split(','))
datasetInFrame["cit"] = city.apply(lambda x: x[0])
datasetInFrame.head()

,id,txt,val,int,cit,dia,date,time
0,line_fdl,"tô tão pensativa ultimamente, nn tô me sentind...",3,3,Rio de Janeiro,6,10/27/2017,0:17:49
1,line_fdl,guilherme só aparece aqui em casa p fazer barb...,5,4,Rio de Janeiro,5,10/26/2017,20:47:35
2,line_fdl,amo cheiro de café pela manhã.. hot_beverage ...,5,4,Rio de Janeiro,5,10/26/2017,9:45:33
3,line_fdl,tô meio aérea,4,2,Rio de Janeiro,5,10/26/2017,1:24:55
4,line_fdl,"tô mega cansada nn sei que cançaso é esse, só ...",2,4,Rio de Janeiro,5,10/26/2017,0:56:21


In [54]:
datasetInFrame["cit"] = datasetInFrame["cit"].apply(lambda x: 1 if x == "Rio de Janeiro" else 2)
datasetInFrame.head()

,id,txt,val,int,cit,dia,date,time
0,line_fdl,"tô tão pensativa ultimamente, nn tô me sentind...",3,3,1,6,10/27/2017,0:17:49
1,line_fdl,guilherme só aparece aqui em casa p fazer barb...,5,4,1,5,10/26/2017,20:47:35
2,line_fdl,amo cheiro de café pela manhã.. hot_beverage ...,5,4,1,5,10/26/2017,9:45:33
3,line_fdl,tô meio aérea,4,2,1,5,10/26/2017,1:24:55
4,line_fdl,"tô mega cansada nn sei que cançaso é esse, só ...",2,4,1,5,10/26/2017,0:56:21


### Criação de  *features*

Após o processamento, definimos como *features* interessantes a serem consideradas como Final de Semana e Madrugada. Intuitivamente, os tweets tendem a ser mais felizes nos finais de semana, mas será que tal característica é de fato relevante? Como uma tentativa de responder a essa questão, foram adicionadas duas colunas de:

1. isWeekend? [dia 6 após às 19:00, dia 7 e 1 até 19:00 - 1 sim, 0 não
2. isMad? [23:00 às 5:00] - 1 sim, 0 não



In [55]:
def isWeekend(day,hour):
    format = '%H:%M:%S'
    hour = datetime.strptime(hour, format)
    limitWeekend = datetime.strptime("19:00:00", format)
    if (day == 6) and (hour >= limitWeekend):
        return 1
    elif (day == 1) and (hour < limitWeekend):
        return 1
    elif day == 7:
        return 1
    else:
        return 0

datasetInFrame['isWeekend'] = pd.Series(np.zeros(len(datasetInFrame)), index=datasetInFrame.index)

for index, row in datasetInFrame.iterrows():
    datasetInFrame.loc[index,"isWeekend"] = isWeekend(row["dia"],row["time"])
datasetInFrame.head()

,id,txt,val,int,cit,dia,date,time,isWeekend
0,line_fdl,"tô tão pensativa ultimamente, nn tô me sentind...",3,3,1,6,10/27/2017,0:17:49,0.0
1,line_fdl,guilherme só aparece aqui em casa p fazer barb...,5,4,1,5,10/26/2017,20:47:35,0.0
2,line_fdl,amo cheiro de café pela manhã.. hot_beverage ...,5,4,1,5,10/26/2017,9:45:33,0.0
3,line_fdl,tô meio aérea,4,2,1,5,10/26/2017,1:24:55,0.0
4,line_fdl,"tô mega cansada nn sei que cançaso é esse, só ...",2,4,1,5,10/26/2017,0:56:21,0.0


In [56]:
def isMad(hour):
    format = '%H:%M:%S'
    hour = datetime.strptime(hour, format)
    initLimit = datetime.strptime("23:00:00", format)
    endLimit = datetime.strptime("05:00:00", format)
    
    if (hour >= initLimit) or (hour < endLimit):
        return 1
    else:
        return 0

    
datasetInFrame['isMad'] = pd.Series(np.zeros(len(datasetInFrame)), index=datasetInFrame.index)

for index, row in datasetInFrame.iterrows():
    datasetInFrame.loc[index,"isMad"] = isMad(row["time"])
datasetInFrame.head()

,id,txt,val,int,cit,dia,date,time,isWeekend,isMad
0,line_fdl,"tô tão pensativa ultimamente, nn tô me sentind...",3,3,1,6,10/27/2017,0:17:49,0.0,1.0
1,line_fdl,guilherme só aparece aqui em casa p fazer barb...,5,4,1,5,10/26/2017,20:47:35,0.0,0.0
2,line_fdl,amo cheiro de café pela manhã.. hot_beverage ...,5,4,1,5,10/26/2017,9:45:33,0.0,0.0
3,line_fdl,tô meio aérea,4,2,1,5,10/26/2017,1:24:55,0.0,1.0
4,line_fdl,"tô mega cansada nn sei que cançaso é esse, só ...",2,4,1,5,10/26/2017,0:56:21,0.0,1.0


### Histogramas do Dataset
Como uma forma de analisar a influência de tais *features* os histogramas abaixo foram gerados.

#### Valência entre Final de Semana (1.0) e Dia de Semana (0.0)
Em termos de números, o dataset possui muito mais tweets classificados em dias semana do que dia de final de semana, entretanto quando dados são convertidos em percentual, apresentam uma sensível diferença. Os tweets em finais de semana tendem a ser mais felizes (sensivelmente)

In [57]:
#Valência em Fim de Semana
datasetInFrame.hist(column='val',by='isWeekend', bins=10)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000B38165FDA0>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x000000B381851518>], dtype=object)

#### Valência na Madrugada (1.0) e Não (0.0)

In [58]:
#Valência na Madrugada
datasetInFrame.hist(column='val',by='isMad', bins=10)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000B381790A20>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x000000B381E7BCF8>], dtype=object)

#### Valência por Cidade

In [59]:
#Valência por Cidade
#datasetInFrame.hist(column='val',by='cit', bins=10)

#### Valência por Intensidade

In [60]:
#Valência por Intensidade
#datasetInFrame.hist(column='val',by='int', bins=10)

#### Valência por Dia

In [61]:
#Valência por Dia
#datasetInFrame.hist(column='val',by='date', bins=10)

### Balanceamento  das amostras
Outra etapa importante do pré-processamento é o balanceamento dos dados, ou seja, realizar uma distribuição mais uniforme de amostras para as classes de valência. Para tornar mais balanceado, foram selecionadas até 100 amostras de cada.

In [62]:
balancedDataset = pd.DataFrame()
limitSamples = 100
for i in datasetInFrame['val'].unique():
    balancedDataset = balancedDataset.append(datasetInFrame[datasetInFrame['val'] == i].iloc[:limitSamples],ignore_index=True)
    
balancedDataset.head()

,id,txt,val,int,cit,dia,date,time,isWeekend,isMad
0,line_fdl,"tô tão pensativa ultimamente, nn tô me sentind...",3,3,1,6,10/27/2017,0:17:49,0.0,1.0
1,line_fdl,joana fez eu me arrumar atoa affff cr,3,5,1,4,10/25/2017,0:45:39,0.0,1.0
2,line_fdl,tô tão sem paciência sem.or,3,5,1,6,10/20/2017,23:25:23,1.0,1.0
3,line_fdl,até gosto de calor mas tá de mais mds scrr,3,4,1,5,10/19/2017,22:36:52,0.0,0.0
4,line_fdl,tô com uma dorzinha d cabeça q ta me dx enjoad...,3,4,1,4,10/18/2017,22:33:22,0.0,0.0


In [63]:
sns.countplot(x='val', data=balancedDataset)

In [64]:
#Seleciona aleatóriamente
dataset = balancedDataset.sample(len(balancedDataset), replace=True)
sns.countplot(x='val', data=dataset)

In [65]:
dataset.head()

,id,txt,val,int,cit,dia,date,time,isWeekend,isMad
304,line_fdl,"sdds sair p sambar, tô precisando..",2,3,1,1,10/22/2017,14:49:21,1.0,0.0
134,FabricioGemada,vai rola o churrasco dos crias,5,3,1,3,10/24/2017,22:50:29,0.0,0.0
565,breendoo18,crlh meu telefone voltoooooou porraaaaaaaaa,7,7,1,7,10/21/2017,14:17:05,1.0,0.0
394,anneray_hd,fico cada dia mais decepcionada e assustada co...,2,5,1,4,10/25/2017,21:13:55,0.0,0.0
120,brunaarvi,ao invés de ficar me frustrando e comentando s...,5,4,1,5,10/19/2017,0:40:39,0.0,1.0


### Pré-processamento dos tweets
Remoção de caracteres especiais e conversão em letras minúsculas.

In [66]:
sentences = [str(sentence).lower().replace("'","").replace(".","").replace(",","").replace('"',"").replace("?","") for sentence in dataset["txt"]]
dataset["txt"] = sentences
dataset.head()

,id,txt,val,int,cit,dia,date,time,isWeekend,isMad
304,line_fdl,sdds sair p sambar tô precisando,2,3,1,1,10/22/2017,14:49:21,1.0,0.0
134,FabricioGemada,vai rola o churrasco dos crias,5,3,1,3,10/24/2017,22:50:29,0.0,0.0
565,breendoo18,crlh meu telefone voltoooooou porraaaaaaaaa,7,7,1,7,10/21/2017,14:17:05,1.0,0.0
394,anneray_hd,fico cada dia mais decepcionada e assustada co...,2,5,1,4,10/25/2017,21:13:55,0.0,0.0
120,brunaarvi,ao invés de ficar me frustrando e comentando s...,5,4,1,5,10/19/2017,0:40:39,0.0,1.0


### Tokenizing

In [67]:
sentencesWithTokens = [nltk.word_tokenize(sentence.lower()) for sentence in dataset["txt"]]
dataset["txt"] = sentencesWithTokens
dataset.head()

,id,txt,val,int,cit,dia,date,time,isWeekend,isMad
304,line_fdl,"[sdds, sair, p, sambar, tô, precisando]",2,3,1,1,10/22/2017,14:49:21,1.0,0.0
134,FabricioGemada,"[vai, rola, o, churrasco, dos, crias]",5,3,1,3,10/24/2017,22:50:29,0.0,0.0
565,breendoo18,"[crlh, meu, telefone, voltoooooou, porraaaaaaaaa]",7,7,1,7,10/21/2017,14:17:05,1.0,0.0
394,anneray_hd,"[fico, cada, dia, mais, decepcionada, e, assus...",2,5,1,4,10/25/2017,21:13:55,0.0,0.0
120,brunaarvi,"[ao, invés, de, ficar, me, frustrando, e, come...",5,4,1,5,10/19/2017,0:40:39,0.0,1.0


### Remoção das StopWords

In [68]:
stops = set(stopwords.words("portuguese"))
#words = ' '.join([w for w in words if not w in stops])

sentencesWithoutStopWords = []
for i,row in dataset.iterrows():
    sentence =  ' '.join([w for w in row["txt"] if not w in stops])
    sentencesWithoutStopWords.append(sentence.strip())
dataset["txt"]=sentencesWithoutStopWords
dataset.head()

,id,txt,val,int,cit,dia,date,time,isWeekend,isMad
304,line_fdl,sdds sair p sambar tô precisando,2,3,1,1,10/22/2017,14:49:21,1.0,0.0
134,FabricioGemada,vai rola churrasco crias,5,3,1,3,10/24/2017,22:50:29,0.0,0.0
565,breendoo18,crlh telefone voltoooooou porraaaaaaaaa,7,7,1,7,10/21/2017,14:17:05,1.0,0.0
394,anneray_hd,fico cada dia decepcionada assustada rumo mund...,2,5,1,4,10/25/2017,21:13:55,0.0,0.0
120,brunaarvi,invés ficar frustrando comentando sobre algo q...,5,4,1,5,10/19/2017,0:40:39,0.0,1.0


## *Features* de Texto

Uma comum abordagem de classificação de texto é a utilização de *Bag of Words*, que fará a contagem da ocorrência de uma palavra nos dados de treino, e em seguida, o *Term Frequency-Inverse Document Frequency* que divide o número de ocorrências de cada palavra em um documento pelo número total de palavras no documento. Como estratégia, escolhemos utilizar Bigrams(ngram), que considerará a ocorrência cada palavra e a anterior

### *Bag Of Words*

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(token_pattern=r"(?u)\b[a-zA-Z]\w+\b",lowercase=True, ngram_range=(1, 2))
datasetTrain = count_vect.fit_transform(dataset["txt"])

###Visualização
#from pprint import pprint
#pprint(count_vect.vocabulary_)

In [70]:
#datasetTrain.todense()
print("Total de features: ", datasetTrain.shape)

Total de features:  (569, 2652)


### *Term Frequency-Inverse Document Frequency*

In [71]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(norm="l2",use_idf=True)
X_tfidf = tfidf_transformer.fit_transform(datasetTrain)

## Separação do dataset
A abordagem definida foi de 70% para treino e 30% para teste

In [72]:
train = datasetTrain[:int(datasetTrain.shape[0] * 0.7)]
trainTarget = dataset['val'][:int(len(dataset['val']) * 0.7)]

test = datasetTrain[:-int(datasetTrain.shape[0] * 0.3)]
testTarget = dataset['val'][:-int(len(dataset['val']) * 0.3)]


## Abordagem com SVM
A abordagem com o SVM foi escolhida, pois ele apresenta bons resultados e por experiências anteriores com o algoritmo.

A implementação utilizada disponível em [scikit-learn](http://scikit-learn.org/stable/index.html) uma biblioteca em Python amplamente utilizada para tarefas de *Machine Learning*.

Parâmetros:
* Kernel = 'rbf', recomendado por algumas referências [1] como uma boa escolha de primeira abordagem. O "poly" teve um desempenho muito ruim e o "linear" apresentou um score alto.
* Gamma = 0, apresentou bons resultados quando "rbf"
* C = 30, apresentou bons resultados quando "rbf", quando 100 aproximou de 98%


In [95]:
#svm = SVC(kernel="linear") # ~ 99 
#svm = SVC(kernel="poly") # ~ 19 
#svm = SVC(kernel='rbf',gamma=0.001, C=100.) # ~ 98
svm = SVC(kernel='rbf',gamma=0.001, C=30.) # ~87

In [96]:
print("Original number of features : %d" % train.shape[1])
svm.fit(train, trainTarget)
predict = svm.predict(test)
print("Score: ",svm.score(test, testTarget))

Original number of features : 2652
Score:  0.84962406015


### Matriz de Confusão

In [97]:
print (pd.crosstab(testTarget, predict, rownames=['Real'], colnames=['Classificado'], margins=True))
#print(confusion_matrix(testTarget, predict, labels = [1, 2, 3,4,5,6,7]))

Classificado   1    2   3   4   5   6  7  All
Real                                         
1             24   16   0   0   0   0  0   40
2              0   80   1   0   0   0  0   81
3              0    3  72   0   0   0  0   75
4              0   13   1  34   2   0  0   50
5              0    4   0   0  71   0  0   75
6              0    8   1   0   2  49  0   60
7              0    6   0   0   1   2  9   18
All           24  130  75  34  76  51  9  399


In [98]:
balancedDataset.head()

,id,txt,val,int,cit,dia,date,time,isWeekend,isMad
0,line_fdl,"tô tão pensativa ultimamente, nn tô me sentind...",3,3,1,6,10/27/2017,0:17:49,0.0,1.0
1,line_fdl,joana fez eu me arrumar atoa affff cr,3,5,1,4,10/25/2017,0:45:39,0.0,1.0
2,line_fdl,tô tão sem paciência sem.or,3,5,1,6,10/20/2017,23:25:23,1.0,1.0
3,line_fdl,até gosto de calor mas tá de mais mds scrr,3,4,1,5,10/19/2017,22:36:52,0.0,0.0
4,line_fdl,tô com uma dorzinha d cabeça q ta me dx enjoad...,3,4,1,4,10/18/2017,22:33:22,0.0,0.0


## Validação

Para validar o classificador foi utilizado um dataset sem labels, o dataset é composto das seguintes colunas:

1. id - Usuário no Twitter
2. txt - O conteúdo do tweet
3. cit - Cidade e País do usuário
4. data - Data e hora do tweet
5. dia - Dia da semana

Total: 4202 amostras x 5 colunas.

O dataset passou pelo mesmo pré-processamento de emoji que o dataset utilizado para treino.

In [99]:
datasetValida = pd.read_csv("demojized_input-emoji.csv", sep="|",quoting=csv.QUOTE_ALL)

### Colocando espaços entre os emojis
sentencesEmojis = [str(sentence).lower().replace(":"," ") for sentence in datasetValida["txt"]]
datasetValida["txt"] = sentencesEmojis

In [100]:
tweets = datasetValida["txt"]
validaCounts = count_vect.transform(tweets)
validaTfidf = tfidf_transformer.transform(validaCounts)

validaPredict = svm.predict(validaTfidf)

for doc, category in zip(tweets, validaPredict):
    #print(doc, " ", category)
    pass

### Valência e Intensidade
A valência foi predita pelo modelo e a intensidade adotada como padrão foi a 4 (neutra).

In [101]:
datasetValida["val"] = pd.Series(validaPredict, index=datasetValida.index)

#Todas as intensidades como 4
intensity = [4]*len(datasetValida)
datasetValida["int"] = pd.Series(intensity, index=datasetValida.index)

datasetValida.head()

,id,txt,cit,data,cit.1,val,int
0,onnabia,apenas um filme lésbico pode me alegrar neste ...,"Rio de Janeiro, Brasil",27/10/2017 00:14,6,2,4
1,onnabia,fodase só queria alguém que me amasse e aceita...,"Rio de Janeiro, Brasil",27/10/2017 00:08,6,2,4
2,onnabia,"já deu pra mim, já tentei de todas as maneiras...","Rio de Janeiro, Brasil",27/10/2017 00:01,6,2,4
3,onnabia,se liga vou fazer um drama,"Rio de Janeiro, Brasil",27/10/2017 00:00,6,2,4
4,onnabia,me sentindo levemente suicida,"Rio de Janeiro, Brasil",26/10/2017 23:57,5,2,4


### Tratamento de dados para saída

In [102]:
datasetValida["txt"] = sentencesEmojis
datasetValida.head()

formato = '%d/%m/%Y %H:%M'
dateTimeConverted = [datetime.strptime(dt, formato).strftime('%m/%d/%y %H:%M') for dt in datasetValida["data"]]
datasetValida["data"] = dateTimeConverted
sentencesValida = [str(sent) for sent in datasetValida["txt"]]
datasetValida["txt"] = sentencesValida

,id,txt,cit,data,cit.1,val,int
0,onnabia,apenas um filme lésbico pode me alegrar neste ...,"Rio de Janeiro, Brasil",10/27/17 00:14,6,2,4
1,onnabia,fodase só queria alguém que me amasse e aceita...,"Rio de Janeiro, Brasil",10/27/17 00:08,6,2,4
2,onnabia,"já deu pra mim, já tentei de todas as maneiras...","Rio de Janeiro, Brasil",10/27/17 00:01,6,2,4
3,onnabia,se liga vou fazer um drama,"Rio de Janeiro, Brasil",10/27/17 00:00,6,2,4
4,onnabia,me sentindo levemente suicida,"Rio de Janeiro, Brasil",10/26/17 23:57,5,2,4
5,onnabia,chapada de zolpidem graças a deus,"Rio de Janeiro, Brasil",10/26/17 23:29,5,2,4
6,onnabia,to lendo aqui freud a interpretação dos sonhos...,"Rio de Janeiro, Brasil",10/26/17 22:38,5,2,4
7,onnabia,“sonhar com borboletas preste mais atenção no...,"Rio de Janeiro, Brasil",10/26/17 22:33,5,2,4
8,onnabia,eu não entendo quem criou que eu sonhar com de...,"Rio de Janeiro, Brasil",10/26/17 22:32,5,2,4
9,onnabia,vcs acreditam que sonhos têm significados?,"Rio de Janeiro, Brasil",10/26/17 22:31,5,2,4


### Histograma das classificações

In [107]:
sns.countplot(x='val', data=datasetValida)

In [106]:
datasetValida.head()

,id,txt,cit,data,cit.1,val,int
0,onnabia,apenas um filme lésbico pode me alegrar neste ...,"Rio de Janeiro, Brasil",10/27/17 00:14,6,2,4
1,onnabia,fodase só queria alguém que me amasse e aceita...,"Rio de Janeiro, Brasil",10/27/17 00:08,6,2,4
2,onnabia,"já deu pra mim, já tentei de todas as maneiras...","Rio de Janeiro, Brasil",10/27/17 00:01,6,2,4
3,onnabia,se liga vou fazer um drama,"Rio de Janeiro, Brasil",10/27/17 00:00,6,2,4
4,onnabia,me sentindo levemente suicida,"Rio de Janeiro, Brasil",10/26/17 23:57,5,2,4


In [105]:
datasetValida.to_csv('output-classificador-com-demoji-rbf-85.csv', ",", index=False)
print("Arquivo gerado: output-classificador-com-dmoji-85.csv ")

Arquivo gerado: output-classificador-com-dmoji-87.csv 


## Conclusões
Foi possível obter uma classificação inicial válida e com bons resultados (em muitos casos "overfitting") a partir do SVM. Avaliando os diferentes parâmetros selecionados, obtivemos os seguintes resultados:

* Kernel Linear - 99% e se apresentando um classificador "otimista"
* Kernel Polynomial - 19% 
* Kernel Radial Basis Function (gamma=0.001, C=100) - 98% e se apresentando um classificador "pessimista"
* Kernel Radial Basis Function (gamma=0.001, c=30) - 87% e classificou apenas as classes 3,4 e 5 (também "pessimista")

Na matriz de confusão do Kernel Radial Basis Function (gamma=0.001, c=30), o modelo classificou muito bem as classes, errando mais na classe de valência 3, indicando uma forte tendência em classificar os tweets como negativos, o que chamamos aqui de “pessimista”. Outro interessante comportamento do classificador com tais parâmetros, é que classificou apenas as classes 3,4 e 5, o que podemos trocar por Negativo, Neutro e Positivo.

Apesar das classificações serem dadas como "pessimistas" no caso do RBF, as classificações se mostraram, de certa forma, coerentes com o conteúdo dos tweets.

Entre as dificuldades e fraquezas, podem ser citadas o processo de extração de *features*, nesse caso, para se aproximar do objetivo da ferramenta proposta no projeto, devemos estudar e adicionar outras *features* a serem consideradas no classificador, tais como final de semana, madrugada, entre outras que julgarmos relacionadas. 
No decorrer da implementação, encontramos algumas dificuldades em considerar os emojis, mas ao final da implementação, eles foram adicionados ao BoW e ao classificador.

Uma questão importante a ser considerada, é de que os tweets possuem muitas gírias, siglas e erros ortográficos. Dessa forma, acreditamos que, como próximos passos, seja uma avaliação mais profunda no impacto disso no classificador e a resolução de tal problema.
Outro ponto importante, como trabalho futuro, é a classificação de intensidade. Na implementação apresentada, apenas a classificação da Valência foi considerada.


## Referências

[1] NLM. Faceli K, Lorena AC, Gama J, Carvalho ACP de LF de. **Inteligência artificial: uma abordagem de aprendizado de máquina**. Rio de Janeiro: LTC, 2011.

[2] [Aprendizado de Máquina com Python](https://iascblog.wordpress.com/2017/03/17/aprendizado-de-maquina-supervisionado-com-python/)

[3] [Código de Exemplo](https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/)

[4] [Cross Validation](http://juliocesarbatista.com/post/Cross-validation-testando-o-desempenho-de-um-classificador/)

[5] [Scikit-learn - Trabalhando com Textos](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)